# 📊 Выгрузка справочника ценных бумаг с MOEX ISS

## 📝 Описание

Этот notebook предназначен для выгрузки данных о ценных бумагах с Московской биржи (MOEX) через API ISS и сохранения их в формат Excel.

## 🚀 Возможности

✅ Получение полного справочника ценных бумаг
✅ Фильтрация по различным критериям (рынок, движок, торговый статус и др.)
✅ Поиск по коду, названию, ISIN и другим параметрам
✅ Сохранение в Excel с возможностью включения данных о торговых площадках

## 📋 Примеры использования

1. **ВЕСЬ справочник** (все бумаги) на русском, без фильтров:
   ```bash
   python dump_iss_securities_to_excel.py --out moex_securities.xlsx
   ```

2. **Только торгуемые фьючерсы** на площадке FORTS:
   ```bash
   python dump_iss_securities_to_excel.py --engine futures --market forts --is-trading 1 --out forts_securities.xlsx
   ```

3. **Поиск по подстроке** 'MOEX' (код/название/ISIN/эмитент/госрег):
   ```bash
   python dump_iss_securities_to_excel.py --q MOEX --out search_moex.xlsx
   ```

In [1]:
# 📦 Импорт необходимых библиотек
import time
from typing import Dict, Any, List

import requests
import pandas as pd

# 🌐 Конфигурация API
BASE = "https://iss.moex.com"
PATH = "/iss/securities.json"
HEADERS = {"User-Agent": "Mozilla/5.0 (iss-securities-dumper)"}
PAGE_LIMIT = 500  # можно менять (MOEX обычно позволяет 100..1000)

print("✅ Библиотеки успешно импортированы")

✅ Библиотеки успешно импортированы


In [2]:
# 🛠️ Вспомогательные функции
def _get_json(params: Dict[str, Any]) -> Dict[str, Any]:
    """Один запрос к /iss/securities.json."""
    url = BASE + PATH
    r = requests.get(url, params=params, headers=HEADERS, timeout=30)
    r.raise_for_status()
    return r.json()


def _table_to_df(section: dict) -> pd.DataFrame:
    """Секция формата {'columns': [...], 'data': [[...], ...]} -> DataFrame."""
    if not section or not section.get("data"):
        return pd.DataFrame()
    cols = section["columns"]
    data = section["data"]
    return pd.DataFrame(data, columns=cols)


def fetch_table_full(iss_only: str, base_params: Dict[str, Any]) -> pd.DataFrame:
    """
    Скачивает ПОЛНУЮ таблицу (постранично) из /iss/securities.json.
    iss_only: 'securities' или 'boards' (допустимые таблицы в этом эндпоинте).
    """
    frames: List[pd.DataFrame] = []
    start = 0
    while True:
        params = dict(base_params)
        params["iss.only"] = iss_only
        params["start"] = start
        params["limit"] = base_params.get("limit", PAGE_LIMIT)
        js = _get_json(params)
        df = _table_to_df(js.get(iss_only, {}))
        if df.empty:
            break
        frames.append(df)
        # если вернулось меньше лимита — следующей страницы нет
        if len(df) < params["limit"]:
            break
        start += params["limit"]
        time.sleep(0.2)  # чуть-чуть подождём, чтобы не спамить
    if frames:
        out = pd.concat(frames, ignore_index=True)
        # на всякий случай уберём дубли
        out = out.drop_duplicates()
        return out
    return pd.DataFrame()

In [3]:
# 📥 Основная функция для получения данных
def fetch_securities_data(q=None, lang="ru", engine=None, market=None, 
                         is_trading=None, group_by=None, group_by_filter=None, 
                         limit=PAGE_LIMIT, include_boards=False):
    """
    Получает данные о ценных бумагах с MOEX ISS API
    """
    base_params: Dict[str, Any] = {"lang": lang, "limit": limit}

    # эти параметры добавляем только если заданы
    if q:
        base_params["q"] = q
    if engine:
        base_params["engine"] = engine
    if market:
        base_params["market"] = market
    if is_trading is not None:
        base_params["is_trading"] = is_trading
    if group_by:
        base_params["group_by"] = group_by
    if group_by_filter:
        base_params["group_by_filter"] = group_by_filter

    # 1) securities
    print("📥 Загружаю таблицу 'securities' ...")
    df_securities = fetch_table_full("securities", base_params)
    print(f"📊 securities: {len(df_securities)} строк")

    # 2) boards (опционально)
    df_boards = pd.DataFrame()
    if include_boards:
        print("📥 Загружаю таблицу 'boards' ...")
        df_boards = fetch_table_full("boards", base_params)
        print(f"📊 boards: {len(df_boards)} строк")
    
    return df_securities, df_boards

In [4]:
# 🧪 Пример использования
print("🚀 Пример получения данных о ценных бумагах")
print("📈 Получение первых 100 торгуемых фьючерсов с площадки FORTS")

# Загрузка данных
df_securities, df_boards = fetch_securities_data(
    engine="futures", 
    market="forts", 
    is_trading=1, 
    limit=100
)

print(f"✅ Получено {len(df_securities)} записей в securities")
if not df_securities.empty:
    print("📋 Первые 5 строк данных:")
    display(df_securities.head())

🚀 Пример получения данных о ценных бумагах
📈 Получение первых 100 торгуемых фьючерсов с площадки FORTS
📥 Загружаю таблицу 'securities' ...
📊 securities: 401 строк
✅ Получено 401 записей в securities
📋 Первые 5 строк данных:


C:\Users\user\AppData\Local\Temp\ipykernel_15880\3077940771.py:42: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  out = pd.concat(frames, ignore_index=True)


,secid,shortname,regnumber,name,isin,is_traded,emitent_id,emitent_title,emitent_inn,emitent_okpo,type,group,primary_boardid,marketprice_boardid
0,AEH6,AED-3.26,None,Фьючерсный контракт AED-3.26,None,1,NaN,None,None,None,futures,futures_forts,RFUD,None
1,AEM6,AED-6.26,None,Фьючерсный контракт AED-6.26,None,1,NaN,None,None,None,futures,futures_forts,RFUD,None
2,AEZ5,AED-12.25,None,Фьючерсный контракт AED-12.25,None,1,NaN,None,None,None,futures,futures_forts,RFUD,None
3,AFH6,AFLT-3.26,None,Фьючерсный контракт AFLT-3.26,None,1,1242.0,"Публичное акционерное общество ""Аэрофлот – рос...",7712040126,29063984,futures,futures_forts,RFUD,None
4,AFZ5,AFLT-12.25,None,Фьючерсный контракт AFLT-12.25,None,1,1242.0,"Публичное акционерное общество ""Аэрофлот – рос...",7712040126,29063984,futures,futures_forts,RFUD,None


In [5]:
# 💾 Сохранение в Excel
def save_to_excel(df_securities, df_boards, filename="moex_securities.xlsx"):
    """
    Сохраняет данные в Excel-файл
    """
    with pd.ExcelWriter(filename, engine="xlsxwriter") as xw:
        df_securities.to_excel(xw, index=False, sheet_name="securities")
        if not df_boards.empty:
            df_boards.to_excel(xw, index=False, sheet_name="boards")
    
    print(f"✅ Данные успешно сохранены в файл: {filename}")

# 📁 Пример сохранения
# save_to_excel(df_securities, df_boards, "moex_securities_example.xlsx")

In [7]:
save_to_excel(df_securities, df_boards, "moex_securities_example.xlsx")

✅ Данные успешно сохранены в файл: moex_securities_example.xlsx


## 📚 Документация по параметрам

| Параметр | Описание | Тип |
|---------|----------|-----|
| `q` | Поиск по части кода/названия/ISIN/эмитента/рег.номеру | str |
| `lang` | Язык (ru или en) | str |
| `engine` | Фильтр по движку (stock, futures, currency, ...) | str |
| `market` | Фильтр по рынку (shares, forts, selt, index, ...) | str |
| `is_trading` | 1 — только торгуемые, 0 — неторгуемые | int (0/1) |
| `group_by` | Группировка (group|type) | str |
| `group_by_filter` | Фильтр по группе (зависит от group_by) | str |
| `limit` | Размер страницы (по умолчанию 500) | int |
| `include_boards` | Также выгрузить таблицу boards | bool |

## 🎯 Примеры значений параметров

**Движки (engine):**
- `stock` - Фондовый рынок
- `futures` - Срочный рынок
- `currency` - Валютный рынок
- `commodity` - Товарный рынок
- `interventions` - Рынок интервенций

**Рынки (market):**
- `shares` - Рынок акций
- `bonds` - Рынок облигаций
- `forts` - Фьючерсы и опционы
- `selt` - Поставочные валютные контракты
- `otc` - ОТС-система
- `ccp` - CCP система клиринга